## CCamera exploring

In [ ]:
import sys, time, os, asyncio, glob

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient


In [ ]:
client = EfdClient('usdf_efd')

In [ ]:
start = Time("2024-05-02T03:00:00", scale='utc')
end = Time("2024-05-02T04:00:00", scale='utc')


In [ ]:
tel = await client.select_time_series("lsst.sal.CCCamera.logevent_endOfImageTelemetry", \
                                            ['imageNumber', 'timestampAcquisitionStart', 'exposureTime', 'private_efdStamp'], \
                                             start, end)
open = await client.select_time_series("lsst.sal.CCCamera.logevent_startShutterOpen", \
                                            ['private_efdStamp'], \
                                             start, end)

close = await client.select_time_series("lsst.sal.CCCamera.logevent_startShutterClose", \
                                            ['private_efdStamp'], \
                                             start, end)
print(len(tel), len(open), len(close))

In [ ]:
for i in range(6):
    print("Telemetry")
    print("Image number", tel['imageNumber'].iloc[i])
    print("Exposure time", tel['exposureTime'].iloc[i])
    print("Index", tel.index[i])
    print("efd_timestamp", Time(tel['private_efdStamp'].iloc[i], format='unix_tai', scale='tai').utc.isot)
    print("timestamp Start", Time(tel['timestampAcquisitionStart'].iloc[i], format='unix_tai', scale='tai').utc.isot)
    print("Open")
    print("Index", open.index[i])
    print("efd_timestamp", Time(open['private_efdStamp'].iloc[i], format='unix_tai', scale='tai').utc.isot)
    print("Close")
    print("Index", close.index[i])
    print("efd_timestamp", Time(close['private_efdStamp'].iloc[i], format='unix_tai', scale='tai').utc.isot)
    print("")

## Cleaning it up

In [ ]:
cam_start = Time("2024-05-02T03:00:00", scale='utc')
cam_end = Time("2024-05-02T04:00:00", scale='utc')

tel = await client.select_time_series("lsst.sal.CCCamera.logevent_endOfImageTelemetry", \
                                            ['imageNumber', 'timestampAcquisitionStart', 'exposureTime'], \
                                             cam_start, cam_end)

tel['close'] = tel['timestampAcquisitionStart'] + tel['exposureTime']
tel['close'] = Time(tel['close'], format='unix_tai', scale='tai').utc.isot
tel['timestampAcquisitionStart'] = Time(tel['timestampAcquisitionStart'], format='unix_tai', scale='tai').utc.isot
tel = tel.set_index('timestampAcquisitionStart')


In [ ]:
for i in range(len(tel)):
    print("Telemetry")
    print("Image number", tel['imageNumber'].iloc[i])
    print("Open", tel.index[i])
    print("Close", tel['close'].iloc[i])
    print("")

## Checking the different timestamps
### All are the same within ms

In [ ]:
az = await client.select_time_series("lsst.sal.MTMount.azimuth", \
                                            ['timestamp', 'private_efdStamp', 'private_kafkaStamp'], \
                                             start, end)
print(len(az))
for i in [0, 1000, 2000]:
    print("Index", az.index[i])
    print("timestamp", Time(az['timestamp'].iloc[i], format='unix_tai', scale='tai').utc.isot)
    print("efd_timestamp", Time(az['private_efdStamp'].iloc[i], format='unix_tai', scale='tai').isot)
    print("kafka_timestamp", Time(az['private_kafkaStamp'].iloc[i], format='unix_tai', scale='tai').utc.isot)
    print()

In [ ]:
az.index[1] - az.index[0]

In [ ]:
camXmax = 8

In [ ]:
mag = int(np.log10(camXmax))
camXmax = (int(camXmax / 10 ** mag) + 1) * 10 ** mag
camXmax